In [ ]:
!pip install qiskit
!pip install pylatexenc
!pip install qiskit-aer
!pip install qiskit-ibm-runtime

In [ ]:
from typing import List, Optional
from qiskit import transpile, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
import warnings
warnings.filterwarnings("ignore")
import math
import numpy as np

from qiskit_ibm_runtime import SamplerV2                    # para simular repeticiones de un experimento
from qiskit_ibm_runtime.fake_provider import FakeManilaV2   # backend falso que simula hardware real (con ruido)

from qiskit_aer import AerSimulator
backend =  AerSimulator(method='statevector')

#Corrección de errores

El error en computación cuántica se refiere al fenómeno por el cual las computadoras cuánticas, a diferencia de las clásicas, no pueden corregir errores copiando datos codificados una y otra vez. Los errores pueden ocurrir debido a varias fuentes, como:

**Ruido:** las perturbaciones ambientales, como las fluctuaciones de temperatura o las interacciones con las moléculas circundantes, pueden hacer que los qubits pierdan su información.

**Decoherencia:** los qubits son extremadamente sensibles, e incluso cambios leves pueden inducir errores.

**Errores de compuerta cuántica:** las imperfecciones en las compuertas cuánticas, que realizan operaciones en los qubits, pueden introducir errores.

Algunas técnicas de mitigación de errores incluyen:

**Códigos de corrección de errores cuánticos:** estos códigos codifican información cuántica en múltiples qubits físicos para formar un "qubit lógico". Esto permite la detección y corrección de errores.

*En este laboratorio seguiremos el método anterior.*

**Algoritmos de corrección de errores:** estos algoritmos, como el *surface code*, detectan y corrigen errores midiendo síndromes (indicadores de errores) y aplicando operaciones de corrección.


## Conceptos previos


**Ruido cuántico**

El ruido cuántico es una fuente fundamental de ruido en los sistemas cuánticos, que surge de la indeterminación inherente de la mecánica cuántica. Es un proceso aleatorio causado por fluctuaciones en la cantidad de fotones que llegan al detector de un punto a otro.

**Coherencia**

La coherencia cuántica se refiere a la capacidad de un sistema de mantener una relación de fase bien definida entre diferentes estados en una superposición. Esta propiedad permite que los qubits existan en una combinación lineal de estados base, lo que permite el paralelismo cuántico y la interferencia, que son fundamentales para la computación cuántica.




#Errores clásicos (digitales)

Supongamos que deseamos enviar un mensaje en términos de 1s y 0s. Incluso en el caso clásico, existe alguna probabilidad de que algún bit cambie, por ejemplo de 0 a 1.

Si la probabilidad de que un bit cambie de valor (i.e. de que haya error) es 0.1, entonces hay una posibilidad de 1 en 10, de que el bit recibido sea incorrecto. Si se envía 0 diez veces, el mensaje podría leerse como 0001000000.

Una de las soluciones más simples es usar alguna repetición. El *bit de datos* 1 puede codificarse como 111, y de la misma manera un 0 se codifica como 000. Cada *bit de datos* ahora se codifica usando tres bits en lugar de solo uno.

Si ahora enviamos 000 y ocurre un error, entonces el receptor podría ver 001. Como el receptor sabe que debería haber recibido 000 o 111, podría deducir que probablemente se envió 000 y se invirtió un solo bit.

¿Qué pasa si ocurren múltiples errores? Nuestro mensaje se convierte en 011 y el destinatario ahora asume que queríamos enviar 111. El error sigue presente. Al usar la repetición, **reducimos la posibilidad** de que eso suceda.

La probabilidad de error se puede obtener de la distribución binomial:

$$P(k)=\begin{pmatrix}n\\k\end{pmatrix}p^k(1-p)^{n-k}.$$


In [ ]:
# probabilidad de encontrar un error en 000
p1 = 0.10

# probabilidad de que un mensaje tenga 2 o 3 errores
p3 = 3 * p1 * p1 * (1 - p1) + p1 * p1 * p1

print("Probabilidad de un error en 000 = {}".format(p1))
print("Probabilidad de dos o más errores en the 000 = {:.4f}".format(p3))

**¿Qué sucede conforme variamos p1?**

Sabiendo cómo puede ayudar la repetición, podemos implementar un programa simple que *codifique* el mensaje anterior. *Deseamos convertir un bit en tres bits*. Después, podríamos escribir un programa simple para *decodificar* el mensaje, tomando los tres bits y convirtiéndolos nuevamente en uno.

## Ejercicio 1

Construyamos un circuito que decodifique un estado de tres bits, utilizando los qubits 0, 1 y 2, en un solo bit en el qubit 4, siguiendo las reglas siguientes:

- 000->0
- 001->0
- 010->0
- 100->0
- 111->1
- 110->1
- 101->1
- 011->1

In [ ]:
def create_decoder(qr: QuantumRegister, cr: ClassicalRegister) -> QuantumCircuit:
# Espere un bit codificado en los primeros 3 qubits y descodifíquelo en el cuarto qubit
# Asegúrese de que los valores de los primeros 3 qubits permanezcan iguales

    qc = QuantumCircuit(qr, cr)
    q0, q1, q2, q3 = qr
    (c0,) = cr

    ####### complete el siguiente espacio #######



    ####################################

    return qc

Veamos cómo se vería nuestra decodificación en el caso en que codificamos 1.

Intente experimentar con diferentes entradas (incluidas las que tienen un error) para ver cómo se comportaría su circuito de corrección de errores en estos casos.

In [ ]:
# Esperamos que un bit se codifique en los primeros 3 qubits y lo descodifiquemos en el cuarto qubitqr = QuantumRegister(4)
qr = QuantumRegister(4)
cr = ClassicalRegister(1)

q0, q1, q2, q3 = qr

# Para codificar un 1. Cámbielos para probar las otras codificaciones.
encoder = QuantumCircuit(qr, cr)
encoder.x(q0)
encoder.x(q1)
encoder.x(q2)

decoder = create_decoder(qr, cr)
qc1 = encoder.compose(decoder)

qc1.measure(q3, cr)

qc1.draw("mpl")

In [ ]:
job = backend.run(qc1,shots=1024)
resultados = job.result()
counts = resultados.get_counts()
counts

#El caso cuántico

Clásicamente, repitiendo un bit podemos atenuar los errores. Cuánticamente, no podemos hacer eso:

1. El Teorema de no clonación.

2. Medir un qubit hará que su estado colapse, lo que significa que debemos tener cuidado al trabajar con qubits entrelazados.


Necesitaremos varios qubits para almacenar la información, además necesitaremos algunos qubits *ancilla* adicionales, que usamos como *estabilizadores*.

La idea es que estos *ancilla* no estén entrelazados con los qubits que almacenan el estado, sin embargo, aún nos dan pistas sobre posibles errores al ser medidos.

Utilizaremos dos conjuntos de qubits, unos utilizados para la codificación y otros utilizados para los estabilizadores.

## Implementación del código de repetición de cambio de bits (*bit-flip*)

In [ ]:
# Configurar un circuito cuántico base para nuestros experimentos.
encoding = QuantumRegister(3)
stabilizer = QuantumRegister(2)

encoding_q0, encoding_q1, encoding_q2 = encoding
stabilizer_q0, stabilizer_q1 = stabilizer

# Resultados de la codificación
results = ClassicalRegister(3)

result_b0, result_b1, result_b2 = results

# Para medir el estabilizador
syndrome = ClassicalRegister(2)

syndrome_b0, syndrome_b1 = syndrome

# El qubit real que está codificado
state = encoding[0]

# Las ancillas utilizadas para codificar el estado
ancillas = encoding[1:]


# Inicialización
def initialize_circuit() -> QuantumCircuit:
    return QuantumCircuit(encoding, stabilizer, results, syndrome)

### Inicialización del qubit

Para proteger un estado cuántico de errores, primero debemos prepararlo.
En general, podemos preparar el estado $$|\Psi_0\rangle \rightarrow |\Psi_1\rangle = (\alpha |0\rangle + \beta |1\rangle)$$
En el circuito siguiente, preparamos el estado físico $$|\Psi_1\rangle = |1\rangle$$

In [ ]:
initial_state = initialize_circuit()

initial_state.x(encoding[0])

initial_state.draw(output="mpl")

### Codificación del qubit

De manera similar al caso clásico, queremos usar la repetición para almacenar el qubit inicial.

Queremos mapear nuestro estado $|\Psi_1\rangle = (\alpha |0\rangle + \beta |1\rangle)$ usando nuestra codificación $U_{en}|\Psi_0\rangle$ al estado $ (\alpha |000\rangle + \beta |111\rangle)$.

Este estado es un estado entrelazado y cuando se mide un qubit, también se conoce el resultado de los otros dos qubits.

Por lo tanto, usaremos CX para crear este estado entrelazado de tres qubits a partir del estado inicial de un qubit:

In [ ]:
# Codificación mediante código de inversión de bits
def encode_bit_flip(qc, state, ancillas):
    qc.barrier(state, *ancillas)
    for ancilla in ancillas:
        qc.cx(state, ancilla)
    return qc


# El circuito que codifica nuestra qubit
encoding_circuit = encode_bit_flip(initialize_circuit(), state, ancillas)

# El circuito incluyendo todas las partes hasta ahora
complete_circuit = initial_state.compose(encoding_circuit)
complete_circuit.draw(output="mpl")

### Preparación de un circuito de decodificación

Para decodificar el estado original, debemos construir un decodificador que haga lo opuesto, es decir, un decodificador $U_{de}|\Psi_0\rangle$ que asigne $ (\alpha |000\rangle + \beta |111\rangle)$ a $|\Psi_1\rangle = (\alpha |0\rangle + \beta |1\rangle)$

Como hace exactamente lo opuesto, podemos invertir nuestro codificador:

$U_{de} = U_{en}^\dagger$

In [ ]:
# Decoding (doing the reverse)
def decode_bit_flip(qc, state, ancillas):
    qc.barrier(state, *ancillas)
    for ancilla in ancillas:
        qc.cx(state, ancilla)
    return qc


decoding_circuit = decode_bit_flip(initialize_circuit(), state, ancillas)

decoding_circuit.draw(output="mpl")

### Medición de estabilizadores

Hemos visto anteriormente que podemos entrelazar al qubit $A$ con el qubit $B$ utilizando un CX con $A$ como objetivo y $B$ como control (si $B$ ya estaba en superposición o entrelazado).

Después podemos desenredarlo nuevamente utilizando otro CX en $A$ como objetivo con $B$ como control (u otro qubit que esté completamente entrelazado con $A$).

Dado que queremos medir los estabilizadores para obtener una indicación de los posibles errores que ocurrieron, es importante que **NO** estén entrelazados con los qubits que codifican el estado.

Necesitamos un número par de compuertas CX aplicadas a cada estabilizador. Para que el estabilizador sea útil, al medirlo debe indicarnos si ocurrió un error de inversión de bits y en cuál de los tres qubits de codificación ocurrió.

### Ejercicio 2

Calcule los bits de síndrome, de modo que se puedan medir para detectar errores de cambio de bit único. Hemos incluido un código que medirá los bits de síndrome y restablecerá los qubits estabilizadores al estado "0".

Hay diferentes formas de hacer esto, así que obtengamos un estabilizador con la codificación más simple posible:

`00` -> No se produjo ningún error

`01` -> Se produjo un error en el qubit 0 (el primer qubit)

`10` -> Se produjo un error en el qubit 1 (el segundo qubit)

`11` -> Se produjo un error en el qubit 2 (el tercer qubit)

*Hint: Los tres qubits de codificación deberían estar perfectamente correlacionados, si no lo están, en uno de ellos se produjo un error de cambio de bit.*

In [ ]:
# Agregue funciones de modo que los bits clásicos se puedan usar para ver qué qubit se invierte en el caso de que se invierta un solo qubit.
# Use 2 bits clásicos para ello.
# 0 = 00 = ningún qubit invertido
# 1 = 01 = primer qubit (qubit 0) invertido
# 2 = 10 segundo qubit (qubit 1) invertido
# 3 = 11 = tercer qubit (qubit 2) invertido

def measure_syndrome_bit(qc, encoding, stabilizer):
    qc.barrier()
    encoding_q0, encoding_q1, encoding_q2 = encoding
    stabilizer_q0, stabilizer_q1 = stabilizer

    ####### complete el código #######




    ####################################

    ####### no modifique lo siguiente #######
    qc.barrier()
    qc.measure(stabilizer, syndrome)
    with qc.if_test((syndrome_b0, 1)):
        qc.x(stabilizer_q0)
    with qc.if_test((syndrome_b1, 1)):
        qc.x(stabilizer_q1)

    return qc


syndrome_circuit = measure_syndrome_bit(initialize_circuit(), encoding, stabilizer)

complete_circuit = initial_state.compose(encoding_circuit).compose(syndrome_circuit)
complete_circuit.draw("mpl")

## Corrección de errores

Ahora podemos construir estabilizadores y, al medirlos, obtenemos los síndromes de error. No sólo queremos obtener indicaciones si se produjo un error, sino que queremos poder corregir los errores.

Utilizaremos *circuitos dinámicos* para utilizar nuestras mediciones de síndrome con el fin de corregir errores potenciales. De manera similar al caso clásico, solo podemos corregir como máximo 1 error; si quisiéramos corregir más, necesitaríamos un código más largo, con más qubits de codificación.

### Ejercicio 3

Corrige los errores según los síndromes medidos.

In [ ]:
# Corrija los errores. Recuerde cómo codificamos los errores arriba
def apply_correction_bit(qc, encoding, syndrome):
    qc.barrier()
    encoding_q0, encoding_q1, encoding_q2 = encoding

    ####### complete el código #######


    #####################################

    return qc


correction_circuit = apply_correction_bit(initialize_circuit(), encoding, syndrome)
complete_circuit = (
    initial_state.compose(encoding_circuit)
    .compose(syndrome_circuit)
    .compose(correction_circuit)
)
complete_circuit.draw(output="mpl")

Lo único que falta ahora es medir los qubits de codificación. Aplicaremos el circuito decodificador antes de medir para recuperar el estado inicial.

Si todo funciona perfectamente y sin errores, bastaría con medir solo nuestro qubit inicial, sin embargo, como no siempre es así, medimos todos los qubits para ver si ha ocurrido algo incorrecto.

In [ ]:
def apply_final_readout(qc, encoding, results):
    qc.barrier(encoding)
    qc.measure(encoding, results)
    return qc


measuring_circuit = apply_final_readout(initialize_circuit(), encoding, results)
complete_circuit = (
    initial_state.compose(encoding_circuit)
    .compose(syndrome_circuit)
    .compose(correction_circuit)
    .compose(decoding_circuit)
    .compose(measuring_circuit)
)
complete_circuit.draw(output="mpl")

Ahora que ya tenemos todo podemos probar si obtenemos el resultado correcto.

Haremos una primera prueba sin errores para asegurarnos de que la implementación fue correcta:

In [ ]:
counts = backend.run(complete_circuit, shots=1000).result().get_counts()
plot_histogram(counts)

Podemos ver que obtenemos los resultados correctos (debería dar `00 001`, ya que inicializamos nuestro qubit inicial en el estado 1).

Como puede ver, los otros qubits utilizados en la codificación están en el estado `0` después del proceso como se esperaba.

Ahora sabemos que nuestro circuito funciona sin ruido. Agreguemos algo de ruido.

Para esto, tomamos un simulador que simula el backend ibm_manila, incluido el ruido:

*Nota: Elegimos este backend aquí, ya que tiene un diseño simple. Analizaremos los diseños con más detalle más adelante.*

In [ ]:
backend = FakeManilaV2()
counts = backend.run(complete_circuit, shots=1000).result().get_counts()
plot_histogram(counts)

Obtendremos algunos resultados erróneos, pero en general, la mayoría de los resultados son correctos. Esto es una buena señal y significa que incluso con ruido nuestro código puede funcionar.

Aun así, esto no nos dice qué tan bueno es nuestro esquema, ya que no tenemos una comparación con el caso sin corrección de errores, así que veamos qué tan bueno seríamos sin los pasos de corrección de errores:

In [ ]:
qc3 = (
    initial_state.compose(encoding_circuit)
    .compose(syndrome_circuit)
    .compose(decoding_circuit)
    .compose(measuring_circuit)
)


backend = FakeManilaV2()
counts = backend.run(qc3, shots=1000).result().get_counts()
plot_histogram(counts)

Podemos ver que los resultados son aproximadamente los mismos, o incluso ligeramente peores, ya que no utilizamos los qubits de codificación después de que se crean.

Cuando usamos estos qubits para los cálculos, normalmente se introducirían algunos errores, este no es el caso aquí.

Por otro lado, la parte de corrección de errores puede introducir errores, ya que también consiste en operaciones que toman tiempo.

Para fines de prueba, construimos un circuito, que introduce algunos errores, pero de manera controlada:

- Queremos introducir errores de inversión de bits, ya que eso es lo que estamos corrigiendo

- Queremos que los errores en los 3 qubits de codificación sean independientes entre sí

- Queremos que podamos elegir qué tan alta es la probabilidad de que se introduzcan errores

- Queremos tener nuestra entrada en porcentaje, y la salida debe ser un circuito que genere errores con esa probabilidad.

### Ejercicio 4
Crea un circuito para agregar ruido como se define anteriormente.

In [ ]:
# Agregue algunos errores como se definió anteriormente (solo agregue errores a los qubits de codificación)
def make_some_noise(qc, encoding, syndrome, error_percentage):
    encoding_q0, encoding_q1, encoding_q2 = encoding
    syndrome_b0, syndrome_b1 = syndrome

    ####### modifique el código #######


    ##################################

    return qc


# Construcción de un circuito con una tasa de error del 10% (para cada uno de los qubits de codificación)
noise_circuit = make_some_noise(initialize_circuit(), encoding, syndrome, 10)
noise_circuit.draw(output="mpl")

Utilice el código a continuación para probar su función y crear un circuito que introduzca una tasa de error del 10%.



In [ ]:
qc4 = (
    initial_state.compose(encoding_circuit)
    .compose(noise_circuit)
    .compose(syndrome_circuit)
    .compose(correction_circuit)
    .compose(decoding_circuit)
    .compose(measuring_circuit)
)


backend = FakeManilaV2()
counts = backend.run(qc4, shots=1000).result().get_counts()
plot_histogram(counts)

Podemos ver que nuestros resultados empeoraron, pero aún obtenemos "001" en la mayoría de los casos.

Ahora hemos creado con éxito nuestro primer código de corrección de errores e incluso lo hemos probado.